# Part 2 of the OGC Challenge

## Downloading data and libraries <a id='step1'></a>

In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
pd.set_option('max_columns', None)

In [3]:
response_data_map = {
    "q136r2"    : "I do not watch pay TV",
    "q136r1"    : "I own a TV but don’t use it",
    "q136r3"    : "I do not want others in my household to watch pay TV",
    "q136r4"    : "Price is too high / don’t want to pay for pay TV",
    "q136r5"    : "I only watch free networks or channels available without subscribing to a Pay TV service",
    "q136r6"    : "My promotion ended",
    "q136r7"    : "I no longer found value in a TV service",
    "q136r8"    : "Service issues/connection reliability",
    "q136r9"    : "Customer service interactions",
    "q136r10"   : "Couldn’t get X1 Operating System",
    "q136r11"   : "Other, specify",
    "q136r12"   : "None of the above",
    "q142r2"    : "Faster speed",
    "q142r1"    : "Good price/good value",
    "q142r3"    : "Better reliability; less downtime",
    "q142r4"    : "Customer service",
    "q142r5"    : "Brand preference (prefer Comcast over other providers)",
    "q142r6"    : "Better devices or functionality (e.g., Wifi router)",
    "q142r7"    : "Access to XFINITY WiFi Hotspots outside my house",
    "q142r8"    : "It is the only service available here",
    "q142r9"    : "Positive experience with Comcast before; had previously",
    "q142r10"   : "Saw ad on TV",
    "q142r11"   : "Received flyer in mail",
    "q142r12"   : "Recommended by retail salesperson",
    "q142r13"   : "Recommended by friends/family",
    "q142r14"   : "Bad experience with other companies",
    "q142r15"   : "Was already installed when moved in",
    "q142r16"   : "Was easy to sign up for the service",
    "q142r17"   : "Comcast offered me an incentive for signing up for service (e.g., gift card, electronic device)",
    "q142r18"   : "Comcast offered me a guaranteed rate for a specific amount of time",
    "q142r19"   : "Other, specify:",
    "q138r1"    : "Amazon Prime paid subscription",
    "q138r2"    : "Hulu free version",
    "q138r3"    : "Hulu Plus paid subscription",
    "q138r4"    : "Netflix",
    "q138r5"    : "YouTube",
    "q138r6"    : "TV channel websites",
    "q138r7"    : "Sling TV paid subscription",
    "q138r8"    : "iTunes",
    "q138r9"    : "Google Play",
    "q138r10"   : "Other, specify",
    "q138r11"   : "Do not currently use any service to stream",
    "q138r12"   : "Don’t Know"
}

In [4]:
data_full = pd.read_excel('/Users/dina/Downloads/Dataset_V2.0.xlsx', sheet_name="Full Wave")
data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 91 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   record        1036 non-null   int64  
 1   B             1036 non-null   object 
 2   C             1036 non-null   int64  
 3   Dr1           1036 non-null   object 
 4   Dr2           1036 non-null   object 
 5   Dr3           1036 non-null   object 
 6   Dr4           1036 non-null   object 
 7   Dr5           1036 non-null   object 
 8   Dr6           1036 non-null   object 
 9   Dr7           1036 non-null   object 
 10  Dnone         1036 non-null   object 
 11  E-Zip Code    1036 non-null   int64  
 12  G             1036 non-null   object 
 13  H             1036 non-null   object 
 14  Hr8oe         18 non-null     object 
 15  L             1036 non-null   object 
 16  Lr10oe        0 non-null      float64
 17  X             935 non-null    float64
 18  noanswerX_r1  935 non-null  

In [5]:
data_full.head()

,record,B,C,Dr1,Dr2,Dr3,Dr4,Dr5,Dr6,Dr7,Dnone,E-Zip Code,G,H,Hr8oe,L,Lr10oe,X,noanswerX_r1,q101c1,q101c2,q101c3,q105r1,q105r2,q110r1,q110r2,q113r1,q113r2,q113r3,q113r4,q113r5,q113r6,q115,q116,q117,q120,q125,q125r8oe,q136r1,q136r2,q136r3,q136r4,q136r5,q136r6,q136r7,q136r8,q136r9,q136r10,q136r11,q136r12,q136r11oe,q138r1,q138r2,q138r3,q138r4,q138r5,q138r6,q138r7,q138r8,q138r9,q138r10,q138r11,q138r12,q138r10oe,q142r1,q142r2,q142r3,q142r4,q142r5,q142r6,q142r7,q142r8,q142r9,q142r10,q142r11,q142r12,q142r13,q142r14,q142r15,q142r16,q142r17,q142r18,q142r19,q142r19oe,q001,q001r5oe,q003,q004,q005,q007,q008
0,18,Female,28,NO TO: 1-17,NO TO: 18-24,25-34,NO TO: 35-44,NO TO: 45-54,NO TO: 55-64,NO TO: 65+,NO TO: None of These Classifications Apply,46574,"Yes, I have a choice of provider",FiOS (Verizon),NaN,XFINITY by Comcast,NaN,120.0,NO TO:,High speed Internet,Pay TV,Landline telephone,7,1 - Not at all satisfied,Three months to less than six months,One year to less than three years,5,4,4,5,6,4,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO TO: Amazon Prime paid subscription,NO TO: Hulu free version,NO TO: Hulu Plus paid subscription,Netflix,YouTube,NO TO: TV channel websites,NO TO: Sling TV paid subscription,NO TO: iTunes,Google Play,"NO TO: Other, specify",NO TO: Do not currently use any service to stream,NO TO: Don’t Know,NaN,NO TO: Good price/good value,NO TO: Faster speed,NO TO: Better reliability; less downtime,NO TO: Customer service,NO TO: Brand preference (prefer Comcast over o...,"Better devices or functionality (e.g., Wifi ro...",NO TO: Access to XFINITY WiFi Hotspots outside...,NO TO: It is the only service available here,NO TO: Positive experience with Comcast before...,Saw ad on TV,NO TO: Received flyer in mail,NO TO: Recommended by retail salesperson,NO TO: Recommended by friends/family,NO TO: Bad experience with other companies,NO TO: Was already installed when moved in,NO TO: Was easy to sign up for the service,NO TO: Comcast offered me an incentive for sig...,NO TO: Comcast offered me a guaranteed rate fo...,"NO TO: Other, specify:",NaN,Townhouse or attached home,NaN,Employed full-time,Graduate degree,Not married but in a similarly-committed relat...,No,"$100,000-$149,999"
1,5,Female,24,NO TO: 1-17,18-24,NO TO: 25-34,NO TO: 35-44,NO TO: 45-54,NO TO: 55-64,NO TO: 65+,NO TO: None of These Classifications Apply,48185,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,50.0,NO TO:,High speed Internet,Pay TV,Landline telephone,NaN,6,NaN,Six months to less than one year,NaN,NaN,NaN,NaN,NaN,NaN,3,8,NaN,No,NaN,NaN,NO TO: I own a TV but don’t use it,I do not watch pay TV,NO TO: I do not want others in my household to...,Price is too high / don’t want to pay for pay TV,NO TO: I only watch free networks or channels ...,NO TO: My promotion ended,I no longer found value in a TV service,NO TO: Service issues/connection reliability,NO TO: Customer service interactions,NO TO: Couldn’t get X1 Operating System,"NO TO: Other, specify",NO TO: None of the above,NaN,Amazon Prime paid subscription,Hulu free version,NO TO: Hulu Plus paid subscription,Netflix,YouTube,NO TO: TV channel websites,NO TO: Sling TV paid subscription,NO TO: iTunes,NO TO: Google Play,"NO TO: Other, specify",NO TO: Do not currently use any service to stream,NO TO: Don’t Know,NaN,NO TO: Good price/good value,Faster speed,NO TO: Better reliability; less downtime,NO TO: Customer service,NO TO: Brand preference (prefer Comcast over o...,"Better devices or functionality (e.g., Wifi ro...",NO TO: Access to XFINITY WiFi Hotspots outside...,NO TO: It is the only service available here,NO TO: Positive experience with Comcast before...,NO TO: Saw ad on TV,NO TO: Received flyer in mail,NO TO: Recommended by retail salesperson,NO TO: Recommended by friends/family,NO TO: Bad experience with other companies,NO TO: Was already installed when moved in,NO TO: Was easy to sign up for the service,NO TO: Comcast offered m

In [6]:
data_full = data_full.drop(columns=['Dr1', 'Dr2', 'Dr3', 'Dr4', 'Dr5', 'Dr6', 'Dr7', 'Dnone', 'noanswerX_r1'])
data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 82 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   record      1036 non-null   int64  
 1   B           1036 non-null   object 
 2   C           1036 non-null   int64  
 3   E-Zip Code  1036 non-null   int64  
 4   G           1036 non-null   object 
 5   H           1036 non-null   object 
 6   Hr8oe       18 non-null     object 
 7   L           1036 non-null   object 
 8   Lr10oe      0 non-null      float64
 9   X           935 non-null    float64
 10  q101c1      1036 non-null   object 
 11  q101c2      1036 non-null   object 
 12  q101c3      1036 non-null   object 
 13  q105r1      666 non-null    object 
 14  q105r2      1036 non-null   object 
 15  q110r1      666 non-null    object 
 16  q110r2      1036 non-null   object 
 17  q113r1      666 non-null    object 
 18  q113r2      666 non-null    object 
 19  q113r3      666 non-null   

In [7]:
mutiple_choice_questions = ['q142','q136','q138']
## Replaceing 'NO TO' with 0, otherwise 1
for question_prefix in mutiple_choice_questions:
    question_results_columns = list(filter(lambda col_name: col_name.startswith(question_prefix) & (col_name.endswith("oe") == False), data_full.columns))
    for col in question_results_columns:
        data_full[col] = data_full[col].fillna('NO TO').apply(lambda x: 0 if str(x).startswith("NO TO") else 1)


## Renaming colums in the followinf format:
# "q136r1 - I own a TV but don’t use it",
columns_names = dict(map(lambda kv: (kv[0], kv[0]+" – "+kv[1]), response_data_map.items()))
data_full.rename(columns = columns_names, inplace = True)



In [8]:
## Adding age range column:
def age_range (C):
    if C <= 17:
        return "1-17"
    if C <= 24:
        return "18-24"
    if C <= 34:
        return "25-34"
    if C <= 44:
        return "35-44"
    if C <= 54:
        return "45-54"
    if C <= 64:
        return "55-64"
    return "65+"
data_full['age_range'] = data_full['C'].apply(age_range)

data_full['age_range'].head()


0    25-34
1    18-24
2    25-34
3    55-64
4    25-34
Name: age_range, dtype: object

In [9]:
## Transforming rate questions:
mark_questions = ['q105r1', 'q105r2',  'q113r1', 'q113r2', 'q113r3', 'q113r4', 'q113r5', 'q113r6', 'q115', 'q116', 'q117']
for col in mark_questions:
    data_full[col] = data_full[col].apply(lambda x: re.sub("[^0-9]", "", str(x))).apply(lambda x: "0" if x == '' else x).astype(int)
    


In [10]:
data_full.head()

,record,B,C,E-Zip Code,G,H,Hr8oe,L,Lr10oe,X,q101c1,q101c2,q101c3,q105r1,q105r2,q110r1,q110r2,q113r1,q113r2,q113r3,q113r4,q113r5,q113r6,q115,q116,q117,q120,q125,q125r8oe,q136r1 – I own a TV but don’t use it,q136r2 – I do not watch pay TV,q136r3 – I do not want others in my household to watch pay TV,q136r4 – Price is too high / don’t want to pay for pay TV,q136r5 – I only watch free networks or channels available without subscribing to a Pay TV service,q136r6 – My promotion ended,q136r7 – I no longer found value in a TV service,q136r8 – Service issues/connection reliability,q136r9 – Customer service interactions,q136r10 – Couldn’t get X1 Operating System,"q136r11 – Other, specify",q136r12 – None of the above,q136r11oe,q138r1 – Amazon Prime paid subscription,q138r2 – Hulu free version,q138r3 – Hulu Plus paid subscription,q138r4 – Netflix,q138r5 – YouTube,q138r6 – TV channel websites,q138r7 – Sling TV paid subscription,q138r8 – iTunes,q138r9 – Google Play,"q138r10 – Other, specify",q138r11 – Do not currently use any service to stream,q138r12 – Don’t Know,q138r10oe,q142r1 – Good price/good value,q142r2 – Faster speed,q142r3 – Better reliability; less downtime,q142r4 – Customer service,q142r5 – Brand preference (prefer Comcast over other providers),"q142r6 – Better devices or functionality (e.g., Wifi router)",q142r7 – Access to XFINITY WiFi Hotspots outside my house,q142r8 – It is the only service available here,q142r9 – Positive experience with Comcast before; had previously,q142r10 – Saw ad on TV,q142r11 – Received flyer in mail,q142r12 – Recommended by retail salesperson,q142r13 – Recommended by friends/family,q142r14 – Bad experience with other companies,q142r15 – Was already installed when moved in,q142r16 – Was easy to sign up for the service,"q142r17 – Comcast offered me an incentive for signing up for service (e.g., gift card, electronic device)",q142r18 – Comcast offered me a guaranteed rate for a specific amount of time,"q142r19 – Other, specify:",q142r19oe,q001,q001r5oe,q003,q004,q005,q007,q008,age_range
0,18,Female,28,46574,"Yes, I have a choice of provider",FiOS (Verizon),NaN,XFINITY by Comcast,NaN,120.0,High speed Internet,Pay TV,Landline telephone,7,1,Three months to less than six months,One year to less than three years,5,4,4,5,6,4,0,0,4,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,0,0,0,1,0,0,0,NaN,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN,Townhouse or attached home,NaN,Employed full-time,Graduate degree,Not married but in a similarly-committed relat...,No,"$100,000-$149,999",25-34
1,5,Female,24,48185,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,50.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,Six months to less than one year,0,0,0,0,0,0,3,8,0,No,NaN,NaN,0,1,0,1,0,0,1,0,0,0,0,0,NaN,1,1,0,1,1,0,0,0,0,0,0,0,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Married,No,"$75,000-$99,999",18-24
2,9,Female,31,33834,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,67.0,High speed Internet,Pay TV,Landline telephone,0,10,NaN,One year to less than three years,0,0,0,0,0,0,6,7,0,No,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,1,0,0,0,0,0,0,NaN,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Single,NaN,"$25,000-$34,999",25-34
3,57,Male,58,97086,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,40.0,High speed Internet,Pay TV,Landline telephone,0,8,NaN,Three years or more,0,0,0,0,0,0,1,6,0,No,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1,0,0,1,1,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,"Other, please specify",Triplex,Employed full-time,College graduate,Single,NaN,"$35,000-$49,999",55-64
4,36,Female,27,60171,"No, I do not have a choice (I am limited to on...",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,41.0,High speed Internet,Pay TV,La

In [11]:
data_full['record'].nunique()

1036

In [12]:
data_wave_2 = pd.read_excel('/Users/dina/Downloads/Dataset_V2.0.xlsx', sheet_name="Wave II")
data_wave_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   record     849 non-null    int64  
 1   H          849 non-null    object 
 2   Hr8oe      16 non-null     object 
 3   L          849 non-null    object 
 4   Lr10oe     0 non-null      float64
 5   X          768 non-null    float64
 6   q105r1     545 non-null    object 
 7   q105r2     849 non-null    object 
 8   q113r1     545 non-null    object 
 9   q113r2     545 non-null    object 
 10  q113r3     545 non-null    object 
 11  q113r4     545 non-null    object 
 12  q113r5     545 non-null    object 
 13  q113r6     545 non-null    object 
 14  q115       304 non-null    object 
 15  q116       304 non-null    object 
 16  q117       545 non-null    object 
 17  q138r1     849 non-null    object 
 18  q138r2     849 non-null    object 
 19  q138r3     849 non-null    object 
 20  q138r4    

In [13]:
data_wave_2['record'].nunique()

849

In [14]:
data_wave_2['wave'] = '2'

In [15]:
data_wave_3 = pd.read_excel('/Users/dina/Downloads/Dataset_V2.0.xlsx', sheet_name="Wave III")
data_wave_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   record     789 non-null    int64  
 1   H          789 non-null    object 
 2   Hr8oe      16 non-null     object 
 3   L          789 non-null    object 
 4   Lr10oe     0 non-null      float64
 5   X          718 non-null    float64
 6   q105r1     496 non-null    object 
 7   q105r2     789 non-null    object 
 8   q113r1     496 non-null    object 
 9   q113r2     461 non-null    object 
 10  q113r3     496 non-null    object 
 11  q113r4     496 non-null    object 
 12  q113r5     496 non-null    object 
 13  q113r6     496 non-null    object 
 14  q115       293 non-null    object 
 15  q116       293 non-null    object 
 16  q117       496 non-null    object 
 17  q138r1     789 non-null    object 
 18  q138r2     789 non-null    object 
 19  q138r3     789 non-null    object 
 20  q138r4    

In [16]:
data_wave_3['record'].nunique()

789

In [17]:
data_wave_3['wave'] = '3'

In [18]:
waves_merged = pd.concat([data_wave_2, data_wave_3], ignore_index=True)
waves_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638 entries, 0 to 1637
Data columns (total 31 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   record     1638 non-null   int64  
 1   H          1638 non-null   object 
 2   Hr8oe      32 non-null     object 
 3   L          1638 non-null   object 
 4   Lr10oe     0 non-null      float64
 5   X          1486 non-null   float64
 6   q105r1     1041 non-null   object 
 7   q105r2     1638 non-null   object 
 8   q113r1     1041 non-null   object 
 9   q113r2     1006 non-null   object 
 10  q113r3     1041 non-null   object 
 11  q113r4     1041 non-null   object 
 12  q113r5     1041 non-null   object 
 13  q113r6     1041 non-null   object 
 14  q115       597 non-null    object 
 15  q116       597 non-null    object 
 16  q117       1041 non-null   object 
 17  q138r1     1638 non-null   object 
 18  q138r2     1638 non-null   object 
 19  q138r3     1638 non-null   object 
 20  q138r4  

In [19]:
mark_questions_merged = ['q105r1', 'q105r2',  'q113r1', 'q113r2', 'q113r3', 'q113r4', 'q113r5', 'q113r6', 'q115', 'q116', 'q117']
for col in mark_questions_merged:
    waves_merged[col] = waves_merged[col].apply(lambda x: re.sub("[^0-9]", "", str(x))).apply(lambda x: "0" if x == '' else x).astype(int)

waves_merged.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638 entries, 0 to 1637
Data columns (total 31 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   record     1638 non-null   int64  
 1   H          1638 non-null   object 
 2   Hr8oe      32 non-null     object 
 3   L          1638 non-null   object 
 4   Lr10oe     0 non-null      float64
 5   X          1486 non-null   float64
 6   q105r1     1638 non-null   int64  
 7   q105r2     1638 non-null   int64  
 8   q113r1     1638 non-null   int64  
 9   q113r2     1638 non-null   int64  
 10  q113r3     1638 non-null   int64  
 11  q113r4     1638 non-null   int64  
 12  q113r5     1638 non-null   int64  
 13  q113r6     1638 non-null   int64  
 14  q115       1638 non-null   int64  
 15  q116       1638 non-null   int64  
 16  q117       1638 non-null   int64  
 17  q138r1     1638 non-null   object 
 18  q138r2     1638 non-null   object 
 19  q138r3     1638 non-null   object 
 20  q138r4  

In [20]:
waves_merged['record'].nunique()

992

In [21]:
records_2_3 = waves_merged['record'].tolist()

In [22]:
full_records = data_full['record'].tolist()

In [23]:
missing_records = list(set(full_records)-set(records_2_3))
missing_records


[6146,
 6019,
 14352,
 6677,
 5272,
 6169,
 5273,
 5531,
 6049,
 36,
 5668,
 6054,
 5678,
 6447,
 5040,
 5431,
 6456,
 5432,
 5559,
 5562,
 5818,
 9533,
 5184,
 5572,
 5838,
 5839,
 5456,
 2385,
 5204,
 5591,
 10332,
 6493,
 6365,
 5347,
 5606,
 5482,
 5738,
 4974,
 5102,
 5742,
 1521,
 370,
 5110,
 5755]

In [24]:
data_full.query('record in @missing_records')

,record,B,C,E-Zip Code,G,H,Hr8oe,L,Lr10oe,X,q101c1,q101c2,q101c3,q105r1,q105r2,q110r1,q110r2,q113r1,q113r2,q113r3,q113r4,q113r5,q113r6,q115,q116,q117,q120,q125,q125r8oe,q136r1 – I own a TV but don’t use it,q136r2 – I do not watch pay TV,q136r3 – I do not want others in my household to watch pay TV,q136r4 – Price is too high / don’t want to pay for pay TV,q136r5 – I only watch free networks or channels available without subscribing to a Pay TV service,q136r6 – My promotion ended,q136r7 – I no longer found value in a TV service,q136r8 – Service issues/connection reliability,q136r9 – Customer service interactions,q136r10 – Couldn’t get X1 Operating System,"q136r11 – Other, specify",q136r12 – None of the above,q136r11oe,q138r1 – Amazon Prime paid subscription,q138r2 – Hulu free version,q138r3 – Hulu Plus paid subscription,q138r4 – Netflix,q138r5 – YouTube,q138r6 – TV channel websites,q138r7 – Sling TV paid subscription,q138r8 – iTunes,q138r9 – Google Play,"q138r10 – Other, specify",q138r11 – Do not currently use any service to stream,q138r12 – Don’t Know,q138r10oe,q142r1 – Good price/good value,q142r2 – Faster speed,q142r3 – Better reliability; less downtime,q142r4 – Customer service,q142r5 – Brand preference (prefer Comcast over other providers),"q142r6 – Better devices or functionality (e.g., Wifi router)",q142r7 – Access to XFINITY WiFi Hotspots outside my house,q142r8 – It is the only service available here,q142r9 – Positive experience with Comcast before; had previously,q142r10 – Saw ad on TV,q142r11 – Received flyer in mail,q142r12 – Recommended by retail salesperson,q142r13 – Recommended by friends/family,q142r14 – Bad experience with other companies,q142r15 – Was already installed when moved in,q142r16 – Was easy to sign up for the service,"q142r17 – Comcast offered me an incentive for signing up for service (e.g., gift card, electronic device)",q142r18 – Comcast offered me a guaranteed rate for a specific amount of time,"q142r19 – Other, specify:",q142r19oe,q001,q001r5oe,q003,q004,q005,q007,q008,age_range
4,36,Female,27,60171,"No, I do not have a choice (I am limited to on...",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,41.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,One year to less than three years,0,0,0,0,0,0,1,6,0,No,NaN,NaN,0,0,0,1,1,0,1,0,0,0,1,0,I hate paying for commercials,1,0,0,0,1,0,0,1,1,0,0,0,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,Single family home,NaN,Homemaker,Some college/trade school,Married,Yes,"$50,000-$74,999",25-34
18,370,Female,71,6441,"Yes, I have a choice of provider",DIRECTV,NaN,XFINITY by Comcast,NaN,373.0,High speed Internet,Pay TV,Landline telephone,10,10,Three years or more,Three years or more,10,10,10,10,10,10,0,0,1,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,NaN,Townhouse or attached home,NaN,Retired,Graduate degree,Married,No,"More than $150,000",65+
64,1521,Female,29,33155,"Yes, I have a choice of provider",U-Verse (AT&T),NaN,XFINITY by Comcast,NaN,250.0,High speed Internet,Pay TV,Landline telephone,5,7,Three years or more,One year to less than three years,3,2,3,4,6,1,0,0,6,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,NaN,Townhouse or attached home,NaN,Employed full-time,College graduate,Married,Yes,"$50,000-$74,999",25-34
102,2385,Male,43,2081,"Yes, I have a choice of provider",DIRECTV,NaN,XFINITY by Comcast,NaN,140.0,High speed Internet,Pay TV,Landline telephone,8,9,Three years or more,One year to less than three years,4,2,7,7,7,6,0,0,7,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,1,0,0,1,0,0,0,NaN,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,NaN,Single family home,NaN,Employed part-time,Some college/trade school,Married,Yes,"$50,000-$74,999",35-44
225,4974,Female,35,1376,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,10.0,High speed Internet,Pay TV,Landline telephone,0,8,NaN,Three m

In [42]:
duplicateRows = waves_merged[waves_merged.duplicated(['record'], keep=False)].sort_values(by='record', ascending=True)
duplicateRows.head(20)
len(duplicateRows)

1292

In [26]:
data_full = data_full.sort_values(by = 'record', ascending = True)
data_full.head()

,record,B,C,E-Zip Code,G,H,Hr8oe,L,Lr10oe,X,q101c1,q101c2,q101c3,q105r1,q105r2,q110r1,q110r2,q113r1,q113r2,q113r3,q113r4,q113r5,q113r6,q115,q116,q117,q120,q125,q125r8oe,q136r1 – I own a TV but don’t use it,q136r2 – I do not watch pay TV,q136r3 – I do not want others in my household to watch pay TV,q136r4 – Price is too high / don’t want to pay for pay TV,q136r5 – I only watch free networks or channels available without subscribing to a Pay TV service,q136r6 – My promotion ended,q136r7 – I no longer found value in a TV service,q136r8 – Service issues/connection reliability,q136r9 – Customer service interactions,q136r10 – Couldn’t get X1 Operating System,"q136r11 – Other, specify",q136r12 – None of the above,q136r11oe,q138r1 – Amazon Prime paid subscription,q138r2 – Hulu free version,q138r3 – Hulu Plus paid subscription,q138r4 – Netflix,q138r5 – YouTube,q138r6 – TV channel websites,q138r7 – Sling TV paid subscription,q138r8 – iTunes,q138r9 – Google Play,"q138r10 – Other, specify",q138r11 – Do not currently use any service to stream,q138r12 – Don’t Know,q138r10oe,q142r1 – Good price/good value,q142r2 – Faster speed,q142r3 – Better reliability; less downtime,q142r4 – Customer service,q142r5 – Brand preference (prefer Comcast over other providers),"q142r6 – Better devices or functionality (e.g., Wifi router)",q142r7 – Access to XFINITY WiFi Hotspots outside my house,q142r8 – It is the only service available here,q142r9 – Positive experience with Comcast before; had previously,q142r10 – Saw ad on TV,q142r11 – Received flyer in mail,q142r12 – Recommended by retail salesperson,q142r13 – Recommended by friends/family,q142r14 – Bad experience with other companies,q142r15 – Was already installed when moved in,q142r16 – Was easy to sign up for the service,"q142r17 – Comcast offered me an incentive for signing up for service (e.g., gift card, electronic device)",q142r18 – Comcast offered me a guaranteed rate for a specific amount of time,"q142r19 – Other, specify:",q142r19oe,q001,q001r5oe,q003,q004,q005,q007,q008,age_range
1,5,Female,24,48185,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,50.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,Six months to less than one year,0,0,0,0,0,0,3,8,0,No,NaN,NaN,0,1,0,1,0,0,1,0,0,0,0,0,NaN,1,1,0,1,1,0,0,0,0,0,0,0,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Married,No,"$75,000-$99,999",18-24
2,9,Female,31,33834,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,67.0,High speed Internet,Pay TV,Landline telephone,0,10,NaN,One year to less than three years,0,0,0,0,0,0,6,7,0,No,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,1,0,0,0,0,0,0,NaN,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Single,NaN,"$25,000-$34,999",25-34
0,18,Female,28,46574,"Yes, I have a choice of provider",FiOS (Verizon),NaN,XFINITY by Comcast,NaN,120.0,High speed Internet,Pay TV,Landline telephone,7,1,Three months to less than six months,One year to less than three years,5,4,4,5,6,4,0,0,4,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,0,0,0,1,0,0,0,NaN,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN,Townhouse or attached home,NaN,Employed full-time,Graduate degree,Not married but in a similarly-committed relat...,No,"$100,000-$149,999",25-34
4,36,Female,27,60171,"No, I do not have a choice (I am limited to on...",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,41.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,One year to less than three years,0,0,0,0,0,0,1,6,0,No,NaN,NaN,0,0,0,1,1,0,1,0,0,0,1,0,I hate paying for commercials,1,0,0,0,1,0,0,1,1,0,0,0,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,Single family home,NaN,Homemaker,Some college/trade school,Married,Yes,"$50,000-$74,999",25-34
3,57,Male,58,97086,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,40

In [27]:
data_zip = pd.read_excel('/Users/dina/Downloads/Dataset_V2.0.xlsx', sheet_name="ZIP")
data_zip = data_zip.rename(columns={'SERVICE_ZIP': 'E-Zip Code'})
data_zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7091 entries, 0 to 7090
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   record      7091 non-null   int64 
 1   E-Zip Code  7091 non-null   int64 
 2   Division    7091 non-null   object
 3   Region      7091 non-null   object
dtypes: int64(2), object(2)
memory usage: 221.7+ KB


In [28]:
data = data_full.merge(data_zip[['E-Zip Code', 'Division', 'Region']], how='left', on='E-Zip Code')
data.head()

,record,B,C,E-Zip Code,G,H,Hr8oe,L,Lr10oe,X,q101c1,q101c2,q101c3,q105r1,q105r2,q110r1,q110r2,q113r1,q113r2,q113r3,q113r4,q113r5,q113r6,q115,q116,q117,q120,q125,q125r8oe,q136r1 – I own a TV but don’t use it,q136r2 – I do not watch pay TV,q136r3 – I do not want others in my household to watch pay TV,q136r4 – Price is too high / don’t want to pay for pay TV,q136r5 – I only watch free networks or channels available without subscribing to a Pay TV service,q136r6 – My promotion ended,q136r7 – I no longer found value in a TV service,q136r8 – Service issues/connection reliability,q136r9 – Customer service interactions,q136r10 – Couldn’t get X1 Operating System,"q136r11 – Other, specify",q136r12 – None of the above,q136r11oe,q138r1 – Amazon Prime paid subscription,q138r2 – Hulu free version,q138r3 – Hulu Plus paid subscription,q138r4 – Netflix,q138r5 – YouTube,q138r6 – TV channel websites,q138r7 – Sling TV paid subscription,q138r8 – iTunes,q138r9 – Google Play,"q138r10 – Other, specify",q138r11 – Do not currently use any service to stream,q138r12 – Don’t Know,q138r10oe,q142r1 – Good price/good value,q142r2 – Faster speed,q142r3 – Better reliability; less downtime,q142r4 – Customer service,q142r5 – Brand preference (prefer Comcast over other providers),"q142r6 – Better devices or functionality (e.g., Wifi router)",q142r7 – Access to XFINITY WiFi Hotspots outside my house,q142r8 – It is the only service available here,q142r9 – Positive experience with Comcast before; had previously,q142r10 – Saw ad on TV,q142r11 – Received flyer in mail,q142r12 – Recommended by retail salesperson,q142r13 – Recommended by friends/family,q142r14 – Bad experience with other companies,q142r15 – Was already installed when moved in,q142r16 – Was easy to sign up for the service,"q142r17 – Comcast offered me an incentive for signing up for service (e.g., gift card, electronic device)",q142r18 – Comcast offered me a guaranteed rate for a specific amount of time,"q142r19 – Other, specify:",q142r19oe,q001,q001r5oe,q003,q004,q005,q007,q008,age_range,Division,Region
0,5,Female,24,48185,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,50.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,Six months to less than one year,0,0,0,0,0,0,3,8,0,No,NaN,NaN,0,1,0,1,0,0,1,0,0,0,0,0,NaN,1,1,0,1,1,0,0,0,0,0,0,0,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Married,No,"$75,000-$99,999",18-24,CENTRAL DIVISION,HEARTLAND REGION
1,9,Female,31,33834,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,67.0,High speed Internet,Pay TV,Landline telephone,0,10,NaN,One year to less than three years,0,0,0,0,0,0,6,7,0,No,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,1,0,0,0,0,0,0,NaN,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Single,NaN,"$25,000-$34,999",25-34,CENTRAL DIVISION,FLORIDA REGION
2,18,Female,28,46574,"Yes, I have a choice of provider",FiOS (Verizon),NaN,XFINITY by Comcast,NaN,120.0,High speed Internet,Pay TV,Landline telephone,7,1,Three months to less than six months,One year to less than three years,5,4,4,5,6,4,0,0,4,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,0,0,0,1,0,0,0,NaN,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN,Townhouse or attached home,NaN,Employed full-time,Graduate degree,Not married but in a similarly-committed relat...,No,"$100,000-$149,999",25-34,CENTRAL DIVISION,CHICAGO REGION
3,36,Female,27,60171,"No, I do not have a choice (I am limited to on...",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,41.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,One year to less than three years,0,0,0,0,0,0,1,6,0,No,NaN,NaN,0,0,0,1,1,0,1,0,0,0,1,0,I hate paying for commercials,1,0,0,0,1,0,0,1,1,0,0,0,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,Single family home,NaN,Homemaker,Some college/trade school,Married,Yes,"$50,000-$74,999",25-34,CENT

In [29]:
data.to_excel("survey.xlsx")  

In [30]:
data['E-Zip Code'].nunique()

1036

In [31]:
data['Region'].nunique()

15

In [32]:
data_116 = data.query('q116 > 5')
data_116.head()

,record,B,C,E-Zip Code,G,H,Hr8oe,L,Lr10oe,X,q101c1,q101c2,q101c3,q105r1,q105r2,q110r1,q110r2,q113r1,q113r2,q113r3,q113r4,q113r5,q113r6,q115,q116,q117,q120,q125,q125r8oe,q136r1 – I own a TV but don’t use it,q136r2 – I do not watch pay TV,q136r3 – I do not want others in my household to watch pay TV,q136r4 – Price is too high / don’t want to pay for pay TV,q136r5 – I only watch free networks or channels available without subscribing to a Pay TV service,q136r6 – My promotion ended,q136r7 – I no longer found value in a TV service,q136r8 – Service issues/connection reliability,q136r9 – Customer service interactions,q136r10 – Couldn’t get X1 Operating System,"q136r11 – Other, specify",q136r12 – None of the above,q136r11oe,q138r1 – Amazon Prime paid subscription,q138r2 – Hulu free version,q138r3 – Hulu Plus paid subscription,q138r4 – Netflix,q138r5 – YouTube,q138r6 – TV channel websites,q138r7 – Sling TV paid subscription,q138r8 – iTunes,q138r9 – Google Play,"q138r10 – Other, specify",q138r11 – Do not currently use any service to stream,q138r12 – Don’t Know,q138r10oe,q142r1 – Good price/good value,q142r2 – Faster speed,q142r3 – Better reliability; less downtime,q142r4 – Customer service,q142r5 – Brand preference (prefer Comcast over other providers),"q142r6 – Better devices or functionality (e.g., Wifi router)",q142r7 – Access to XFINITY WiFi Hotspots outside my house,q142r8 – It is the only service available here,q142r9 – Positive experience with Comcast before; had previously,q142r10 – Saw ad on TV,q142r11 – Received flyer in mail,q142r12 – Recommended by retail salesperson,q142r13 – Recommended by friends/family,q142r14 – Bad experience with other companies,q142r15 – Was already installed when moved in,q142r16 – Was easy to sign up for the service,"q142r17 – Comcast offered me an incentive for signing up for service (e.g., gift card, electronic device)",q142r18 – Comcast offered me a guaranteed rate for a specific amount of time,"q142r19 – Other, specify:",q142r19oe,q001,q001r5oe,q003,q004,q005,q007,q008,age_range,Division,Region
0,5,Female,24,48185,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,50.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,Six months to less than one year,0,0,0,0,0,0,3,8,0,No,NaN,NaN,0,1,0,1,0,0,1,0,0,0,0,0,NaN,1,1,0,1,1,0,0,0,0,0,0,0,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Married,No,"$75,000-$99,999",18-24,CENTRAL DIVISION,HEARTLAND REGION
1,9,Female,31,33834,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,67.0,High speed Internet,Pay TV,Landline telephone,0,10,NaN,One year to less than three years,0,0,0,0,0,0,6,7,0,No,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,1,1,1,0,0,0,0,0,0,NaN,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,Apartment building,NaN,Employed full-time,College graduate,Single,NaN,"$25,000-$34,999",25-34,CENTRAL DIVISION,FLORIDA REGION
3,36,Female,27,60171,"No, I do not have a choice (I am limited to on...",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,41.0,High speed Internet,Pay TV,Landline telephone,0,6,NaN,One year to less than three years,0,0,0,0,0,0,1,6,0,No,NaN,NaN,0,0,0,1,1,0,1,0,0,0,1,0,I hate paying for commercials,1,0,0,0,1,0,0,1,1,0,0,0,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,Single family home,NaN,Homemaker,Some college/trade school,Married,Yes,"$50,000-$74,999",25-34,CENTRAL DIVISION,CHICAGO REGION
4,57,Male,58,97086,"Yes, I have a choice of provider",Do not have pay TV service at home,NaN,XFINITY by Comcast,NaN,40.0,High speed Internet,Pay TV,Landline telephone,0,8,NaN,Three years or more,0,0,0,0,0,0,1,6,0,No,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1,0,0,1,1,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,"Other, please specify",Triplex,Employed full-time,College graduate,Single,NaN,"$35,000-$49,999",55-64,WEST DIVISION,PORTLAND REGION
12,261,Female,49,17050,"Yes, I have a choice of

In [33]:
#pd.options.display.float_format = '{:.1%}'.format

In [34]:
statistic_columns = ['age_range', 'B', 'q001', 'q003', 'q004', 'q005', 'q007', 'q008' ]
for col in statistic_columns:
   result=data.groupby([col]).agg({'record': 'count'})
   result['%_of_total'] = result['record'] / len(data)
   result.style.format("{:.1%}")
   result = result.sort_values(by = '%_of_total',ascending = False)
   display(result)
   

,record,%_of_total
age_range,,
45-54,202,0.194981
25-34,189,0.182432
35-44,184,0.177606
55-64,175,0.168919
65+,170,0.164093
18-24,116,0.111969


,record,%_of_total
B,,
Female,518,0.5
Male,518,0.5


,record,%_of_total
q001,,
Single family home,729,0.703668
Apartment building,180,0.173745
Townhouse or attached home,93,0.089768
"Other, please specify",19,0.018340
Apartment in someone's home,15,0.014479


,record,%_of_total
q003,,
Employed full-time,428,0.413127
Retired,207,0.199807
Employed part-time,133,0.128378
Homemaker,119,0.114865
Unemployed,92,0.088803
Student,48,0.046332
Prefer not to say,9,0.008687


,record,%_of_total
q004,,
Some college/trade school,322,0.310811
College graduate,296,0.285714
Completed high school,176,0.169884
Graduate degree,171,0.165058
Some graduate work,53,0.051158
Some high school,11,0.010618
8th grade or less,4,0.003861
Prefer not to say,3,0.002896


,record,%_of_total
q005,,
Married,522,0.503861
Single,233,0.224903
Divorced/separated,122,0.117761
Not married but in a similarly-committed relationship,119,0.114865
Widowed,36,0.034749
Prefer not to say,4,0.003861


,record,%_of_total
q007,,
No,493,0.475869
Yes,320,0.308880
Prefer not to say,10,0.009653


,record,%_of_total
q008,,
"$50,000-$74,999",224,0.216216
"$75,000-$99,999",156,0.150579
"$35,000-$49,999",147,0.141892
"$100,000-$149,999",112,0.108108
"$25,000-$34,999",108,0.104247
"Less than $15,000",96,0.092664
"$15,000-$24,999",93,0.089768
Prefer not to say,53,0.051158
"More than $150,000",43,0.041506


In [35]:
statistic_columns = ['age_range', 'B', 'q001', 'q003', 'q004', 'q005', 'q007', 'q008' ]
for col in statistic_columns:
   result = data_116.groupby([col]).agg({'record': 'count'})
   result['%_of_total'] = (result['record'] / len(data_116))
   
   result = result.sort_values(by = '%_of_total',ascending = False)
   display(result)

,record,%_of_total
age_range,,
25-34,34,0.295652
35-44,26,0.226087
45-54,18,0.156522
18-24,17,0.147826
55-64,11,0.095652
65+,9,0.078261


,record,%_of_total
B,,
Female,64,0.556522
Male,51,0.443478


,record,%_of_total
q001,,
Single family home,59,0.513043
Apartment building,41,0.356522
Townhouse or attached home,10,0.086957
"Other, please specify",3,0.026087
Apartment in someone's home,2,0.017391


,record,%_of_total
q003,,
Employed full-time,48,0.417391
Homemaker,24,0.208696
Employed part-time,13,0.113043
Retired,12,0.104348
Unemployed,11,0.095652
Student,7,0.060870


,record,%_of_total
q004,,
College graduate,49,0.426087
Some college/trade school,31,0.269565
Completed high school,15,0.130435
Graduate degree,15,0.130435
Some graduate work,4,0.034783
8th grade or less,1,0.008696


,record,%_of_total
q005,,
Married,48,0.417391
Single,28,0.243478
Not married but in a similarly-committed relationship,22,0.191304
Divorced/separated,16,0.139130
Widowed,1,0.008696


,record,%_of_total
q007,,
No,45,0.391304
Yes,37,0.321739


,record,%_of_total
q008,,
"$35,000-$49,999",21,0.182609
"$50,000-$74,999",20,0.173913
"$75,000-$99,999",18,0.156522
"$25,000-$34,999",16,0.139130
"$15,000-$24,999",15,0.130435
"Less than $15,000",15,0.130435
"$100,000-$149,999",6,0.052174
"More than $150,000",2,0.017391
Prefer not to say,2,0.017391


In [36]:
duplicateRows.to_excel('in_both_waves.xlsx')

In [37]:
waves_merged.to_excel('merged_waves.xlsx')

In [38]:
data.groupby(['H']).agg({'record': 'nunique'})

,record
H,
Century Link,4
DIRECTV,436
DISH Network,188
Do not have pay TV service at home,370
FiOS (Verizon),4
"Other, specify:",18
U-Verse (AT&T),16


In [39]:
data['q136r1 – I own a TV but don’t use it'].sum()

15

In [40]:
waves_merged.groupby('wave').agg({'q116': 'mean'})

,q116
wave,
2,1.387515
3,1.437262


In [41]:
waves_merged.groupby('wave').agg({'q105r2': 'mean'})

,q105r2
wave,
2,6.948174
3,7.231939
